In [ ]:
# author: Keerthi Sravan Ravi
# date: 21/12/2017

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import keras
import keras.backend as kb
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, Conv2DTranspose, Flatten, Reshape, BatchNormalization, MaxPool2D
from keras.optimizers import adam
from keras.initializers import Constant, glorot_uniform
from keras.utils import plot_model
import sklearn.model_selection
import scipy.misc
%matplotlib inline

## Data loading and saving methods
---

#### 1. load_data_from_disk():
- Load spiral k-space trajectories (x data) from folder `imagenet_kspace`
- Load reconstructed images (y data) from folder `imagenet_orig`

#### 2. save_data_as_npy(arr_path_dict)
For faster access to data, we save the data as `.npy` files.

#### 3. load_data_from_npy(*paths)
In case we have already saved data as `.npy` files, we load the same, as opposed to loading data from `load_data()`.

#### 4. gen_train_test_data(x, y)
Split data into training and testing sets, of proportion:
- Train: 95%
- Test: 5%

In [ ]:
def load_data_from_disk():
    """
    Load x and y data from 'imagenet_64_orig' and 'imagenet_64_kspace'.
    Typically, these two folders should be in the same folder as this code.
    """
    
    root_path = os.getcwd()
    # Find data from 'imagenet_64_orig' and 'imagenet_64_kspace'
    src_imagenet_orig = os.path.join(root_path, 'imagenet_64_orig')
    src_imagenet_kspace = os.path.join(root_path, 'imagenet_64_kspace')

    all_folders = ['animals','artefact','flora','fungus','geo','people', 'sports1']
    print('Folders to load from: {}'.format(all_folders))
    x, y = [], []

    for curr_folder in all_folders:
        print('Currently in folder \'{}\''.format(curr_folder))
        img_folder = os.path.join(src_imagenet_orig, curr_folder)
        kspace_folder = os.path.join(src_imagenet_kspace, curr_folder)

        num_files = len(os.listdir(kspace_folder))
        for i in range(1, num_files):
            img_file = str(i) + '.jpg'
            kspace_file = str(i) + '.npy'

            img_path = os.path.join(img_folder, img_file)
            kspace_path = os.path.join(kspace_folder, kspace_file)

            img = scipy.misc.imread(img_path)
            img_size = img.size
            
            # Skip loading null image
            # If more than 90% of the image is white pixels, then it is a null image
            null_pixels = len(np.where(img >= 255)[0])
            if null_pixels >= 0.9 * img_size:
                continue        

            kspace = np.load(kspace_path)
            # Normalize k-space data
            kspace = 127 * kspace/np.amax(np.abs(kspace))
            kspace_real = np.real(kspace)
            kspace_imag = np.imag(kspace)
            # Convert to int16 to save memory
            kspace = np.stack((kspace_real, kspace_imag), axis=2).astype('int16')

            x.append(kspace)
            y.append(img)

    print('Done')
    
def save_data_as_npy(arr_path_dict):
    """
    Save data to .npy files.
    
    Parameters:
    arr_path_dict : dict
        Key-value pairs of the array to be saved, and the save paths
    """
    
    for item in arr_path_dict.keys():
        np.save(arr_path_dict[item], item)
        
def load_data_from_npy(*paths):
    """
    Load data from .npy files.
    
    Parameters:
    paths : list
        List of paths to load .npy arrays from.

    Returns:
    An array of the .npy files that were loaded from the specified paths.
    """
    
    arrs = []
    for _path in paths:
        print('Loading {}'.format(_path))
        arrs.append(np.load(_path))
    
    print('Done.')
    return arrs

def gen_train_test_data(x, y):
    """
    Generate 95% training and 5% testing data.
    
    Parameters:
    x : ndarray
        X data
    y : ndarray
        Y data
        
    Returns:
    A tuple of x_train, x_test, y_train and y_test
    """
    
    x = x.reshape(x.shape[0], -1)
    y = y.reshape(y.shape[0], -1)
    x_tr, x_ts, y_tr, y_ts = sklearn.model_selection.train_test_split(np.array(x), np.array(y), train_size=0.95)

    print('Training dataset contains {} images'.format(len(x_tr)))
    print('Testing dataset contains {} images'.format(len(y_ts)))
    
    return x_tr, x_ts, y_tr, y_ts

In [ ]:
# Define the callback for normalizing losses
class NormalizeLosses_Callback(keras.callbacks.Callback):
    def __init__(self):
        self.max_loss = 0
        self.all_losses = []
    
    def on_epoch_end(self, epoch, logs={}):
        # We want to record the maximum loss across epochs in this run to normalize losses    
        loss = logs.get('loss')
        self.max_loss = loss if loss > self.max_loss else self.max_loss
        self.all_losses = self.all_losses.append(loss / self.max_loss)
        return

## Create, save and load Keras models
---

#### 1. get_keras_model(return_modified_model=False)
Returns the original Keras Sequential model.

#### 2. save_keras_model_to_disk(model, path)
Save Keras model to disk.

#### 3. load_keras_model_from_disk(path)
Load Keras model from path

In [ ]:
def get_keras_model(return_modified_model=False):
    """
    Parameters:
    return_modified_model : boolean
        Boolean flag to indicate whether the model returned is original or modified.
        
    Returns:
    Keras Sequential model with the following layers, in order:
    - Dense (64 * 64) X 3
    - Reshape
    - { Conv2D(5, 64)
    - BatchNormalization } X 3 (4 if return_modified_model is True)
    - Conv2DTranspose(5, 64)
    - BatchNormalization
    - Flatten
    """
    
    # Clear current Keras session, and create a Sequential model
    kb.clear_session()
    model = Sequential()

    # Constant bias = 0.05, and
    # Truncated normal filter with std = 0.05
    bias_initializer = Constant(value=0.1)
    kernel = glorot_uniform()

    size = 64 * 64
    
    # Fully connected layers 1, 2 and 3
    model.add(Dense(size, use_bias=True, activation='relu', bias_initializer=bias_initializer,
                                 input_shape=(32480,)))
    model.add(Dense(size, use_bias=True, activation='relu', bias_initializer=bias_initializer))
    model.add(Dense(size, use_bias=True, activation='relu', bias_initializer=bias_initializer))

    # Reshape the outputs so far to prepape for convolutional processing
    model.add(keras.layers.Reshape((64, 64, 1)))

    # Convolution layers 1, 2 and 3 (and 4 if return_modified_model=True)
    filter_size = 5
    num_filters = 64
    model.add(Conv2D(num_filters, filter_size, kernel_initializer=kernel, activation='relu',
                            use_bias=True, bias_initializer=bias_initializer, padding='same'))
    model.add(BatchNormalization(center=True, scale=True))
    model.add(Conv2D(num_filters, filter_size, kernel_initializer=kernel, activation='relu',
                            use_bias=True, bias_initializer=bias_initializer, padding='same'))
    model.add(BatchNormalization(center=True, scale=True))
    model.add(Conv2D(num_filters, filter_size, kernel_initializer=kernel, activation='relu',
                            use_bias=True, bias_initializer=bias_initializer, padding='same'))
    model.add(BatchNormalization(center=True, scale=True))
    if return_modified_model:
        model.add(Conv2D(num_filters, filter_size, kernel_initializer=kernel, activation='relu',
                                use_bias=True, bias_initializer=bias_initializer, padding='same'))
        model.add(BatchNormalization(center=True, scale=True))

    # Deconvolution layer 1
    num_deconv_filters = 1
    model.add(Conv2DTranspose(num_deconv_filters, filter_size, kernel_initializer=kernel, activation='relu',
                        use_bias=True, bias_initializer=bias_initializer, padding='same'))
    model.add(BatchNormalization(center=True, scale=True))

    # Flatten the outputs
    model.add(Flatten())

    # Define optimizer, instantiate loss-callback, and compile
    model.compile(optimizer=adam(lr=5e-3), metrics=['acc'], loss='mean_squared_error')
    
    return model

def save_keras_model_to_disk(model, path):
    """
    Save Keras model to disk.
    
    Parameters:
    model : Sequential
        Keras Sequential model to save.
    path : str
        Path to save Keras model to.
    """
    save_path = os.path.join(path, 'keras_model.h5')
    model.save(save_path)
    
def load_keras_model_from_disk(path):
    """
    Load Keras model from disk.
    
    Parameters:
    path : str
        Path to load Keras model from.
    """
    return load_model(path)

## Plot predictions
---
Run the network on, say, 12 k-space data samples (from `x_testing`). Plot the reconstructions versus the ground truth (from `y_testing`).

In [ ]:
def plot_predictions_vs_truth(model, x_ts, y_ts):
    """
    Plot 12 predictions (x_testing) and ground truths (y_testing).
    
    Parameters:
    model : Sequential
        A Keras Sequential model used for predictions.
    x_ts : ndarray
        X testing data 
    y_ts : ndarray
        Y testing data
    num_predictions : int
        Number of predictions to generate and plot
    """
    
    # K-space samples are 280 samples-58 shot, complex numbers
    kspace_size = 280 * 58 * 2
    # Number of predictions
    num_predictions = 12
    # Random index of sample to predict
    starting_index = np.random.randint(len(x_ts) - num_predictions)
    plot_index = 1
    plt.figure(num=1, figsize=(14, 6))
    plt.suptitle('Reconstructions', fontsize=16)
    for x in x_ts[starting_index : starting_index + num_predictions]:
        x = x.reshape(-1, kspace_size)
        y_pred = model.predict(x)
        y_pred = y_pred.reshape((64, 64))
        # Plot 2 rows, 6 columns
        plt.subplot(2, 6, plot_index)
        plt.imshow(y_pred)
        plot_index+=1

    img_flat_shape = (64, 64)
    plot_index = 1
    plt.figure(num=2, figsize=(14, 6))
    plt.suptitle('Ground truths', fontsize=1)
    for y in y_ts[starting_index : starting_index + num_predictions]:
        y = y.reshape(img_flat_shape)
        # Plot 2 rows, 6 columns
        plt.subplot(2, 6, plot_index)
        plt.imshow(y)
        plot_index+=1

## Running the network
---
Now, run everything that we've made!

In [ ]:
# Load data and generate training and testing sets
x, y = load_data_from_npy('x.npy', 'y.npy')
x_tr, x_ts, y_tr, y_ts = gen_train_test_data(x, y)

# Intantiate a normalized-loss callback, and get a Keras model
loss_callback = NormalizeLosses_Callback()
model = get_keras_model()
print(model.summary())

# Fit the model for 50 epochs, passing the normalzied-loss callback we just created
model.fit(x_tr, y_tr, batch_size=32, epochs=1, validation_data=(x_ts, y_ts), callbacks=[loss_callback])

In [ ]:
# Plot 12 predictions and their ground truths
plot_predictions_vs_truth(model, x_ts, y_ts)